In [0]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import json
import sys
import traceback
import os
import datetime
from collections import OrderedDict
import time
import json
from shutil import copyfile
import logging
from pyspark.sql.functions import *
from pyspark.sql.types import *
# Custom Library

sys.path.insert(0, '/dbfs/mnt/mountdatalake/AZ_IDFCodebase/conf/')

import utilsShared
from customException import configNotFoundError,uncatchException
from gen_audit_entry import *


# Read the env Config Properties
start_time = str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
envConfig = utilsShared.getEnvConf(0, dbutils)


envConfig['sourceName'] = dbutils.widgets.get('prm_act_src_name')
envConfig['tableName'] = dbutils.widgets.get('prm_act_file_prefix')
envConfig["batchDate"]=dbutils.widgets.get("prm_act_batchdate")
envConfig['loadDate'] = ''
u_name=dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
envConfig['load_userid'] = u_name[0:u_name.find('@')]
run_id = dbutils.widgets.get('prm_act_runid')
envConfig['prcs_runid'] =  run_id[:32]

sys.path.insert(0, envConfig["data_config_stream_path"])
sys.path.insert(0, envConfig["process_files_stream_path"])

from streamingMetadataManager import streamingMetadataManager
metadata_obj = streamingMetadataManager()

local_log_file_name = envConfig["local_log_path"] + envConfig['sourceName']  + "_" + str(envConfig["tableName"]).replace("/","_") + "_" +str(datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d_%H%M%S')) + '.log'
envConfig['local_log_file_name'] = local_log_file_name     
log_file_name = local_log_file_name.replace(envConfig["local_log_path"], envConfig["log_path"] + "/"  + envConfig['sourceName']+ "/"  + str(envConfig["tableName"]) + "/"+ str(datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d')) + "/")
envConfig['log_file_name'] = log_file_name
log_file_folder = "/".join(log_file_name.split("/")[:-1])
os.makedirs(envConfig["local_log_path"], exist_ok=True)
logger = utilsShared.getFormattedLogger("smart_ingestion_driver", local_log_file_name)
logger.info('Performing Stream Ingestion Activity')

exit_doc = {
      "JSON": "NA",
      "PRCS_NAME": "NA",
      "ID":"NA",
      "RUNID":"NA",
      "NAME":"NA",
      "STREAM_TIMESTAMP":"",
      "BATCHID":"NA",
      "FOREACHBATCH":"NA",
      "STAGE":"NA",
      "STATUS":"NA",
      "STATUS_DESC":"NA",      
      "JOB_START_TIME":start_time,
      "JOB_END_TIME":"",
      "TRIGGER_TYPE":"NA",
      "LOG_PATH":log_file_name,
      "SOURCE_TYPE":"NA",
      "SOURCE_NAME":"NA",
      "SOURCE_DESCRIPTION":"NA",
      "START_OFFSET":"NA",
      "END_OFFSET":"NA",
      "SINK_DESCRIPTION":"NA",
      "CHECKPOINT_PATH":"NA",
      "RAW_PATH":"NA",
      "RAW_ROW_COUNT":"0",
      "RAW_COL_COUNT":"0",
      "CURATED_PATH":"NA",
      "CURATED_ROW_COUNT":"0",
      "CURATED_COL_COUNT":"0",
      "ENRICH_PATH":"NA",
      "ENRICH_ROW_COUNT":"0",
      "ENRICH_COL_COUNT":"0"
}

try:
  if envConfig["loadDate"].strip() == "" :
        envConfig["loadDate"] = str(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
        logger.info("Since load date is not provided, considering load datetime as current/Execution timestamp")
  else:
        loadDate_object = datetime.datetime.strptime(envConfig["loadDate"], "%Y%m%d")
        if date.today() == loadDate_object.date():
            envConfig["loadDate"] = str(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
            logger.info("Since load date & execution date is same , load date/time considered as : " + envConfig["loadDate"])
        else:
            envConfig["loadDate"] = loadDate_object.strftime("%Y%m%d%H%M%S")
            logger.info("Since load date & execution date is different , load date/time used as provided : " + envConfig["loadDate"]) 
  
  suffixConf = "streaming"
  jsonFileName = str(envConfig["sourceName"])+ "_" + str(envConfig["tableName"]) + "_" + suffixConf + ".json"
  dataProcessSourceConfig = envConfig["data_config_stream_path"] + jsonFileName
  logger.info(dataProcessSourceConfig)
  exit_doc['JSON'] = jsonFileName
  dataprocess_config_file=None
  if os.path.isfile(dataProcessSourceConfig):
    dataprocess_config_file = open(str(dataProcessSourceConfig),'r').read()
    config = json.loads(dataprocess_config_file)
    logger.info('Reading config from: ' + str(dataProcessSourceConfig))

  else:
    logger.info('Source Config is not available '+ str(dataProcessSourceConfig) + ', Please place the config file in order to execute the job')
    try:
      raise configNotFoundError(dataProcessSourceConfig)        
    except configNotFoundError as cfe:
      audit_doc = OrderedDict()
      audit_rec=DriverEntryStreaming('streaming',envConfig['sourceName'],envConfig['prcs_runid'],'Failed',cfe,start_time,exit_doc['JSON'],log_file_name)
      audit_doc,exit_doc=utilsShared.gen_audit_dict_streaming(audit_doc,exit_doc,audit_rec,cfe)
      metadata_obj.insert_StreamingauditRecord(dbutils,envConfig,spark,audit_doc)
      exit_doc["EXIT_CODE"]=0
    os.makedirs(log_file_folder, exist_ok=True)
    copyfile(local_log_file_name, log_file_name)
    dbutils.notebook.exit({'AUDIT_RECORDS': exit_doc})
    
  conf = 0
  
  # Adding Common data conf process to envConfig
  if 'S0-common-conf' in config:  # config.has_key("M0-common-conf"):
      envConfig.update(config['S0-common-conf'])
      config.pop('S0-common-conf')
  TABLE_DF = 0
  for key in sorted(config.keys()):
        logger.info('Processing ' + key)
        val = envConfig.copy()
        val.update(config[key])
        typeValue = val['type']
                                        
        if 'NONE' != typeValue and 'D999-alert-monitor-process'!= key:
            logger.debug(typeValue)
            module = __import__(typeValue)
            function = getattr(module, 'process')
            TABLE_DF = function(conf,val, TABLE_DF,dbutils,exit_doc)
            if TABLE_DF == 0:
              break
  
  try:
      logger.info("Exit dict: " + str({'AUDIT_RECORDS': exit_doc, 'EXIT_CODE': exit_doc['EXIT_CODE'], 'ERROR_CODE': exit_doc['ERROR_CODE']}))
      os.makedirs(log_file_folder, exist_ok=True)
      copyfile(local_log_file_name, log_file_name)
      #dbutils.notebook.exit({'AUDIT_RECORDS': exit_doc, 'EXIT_CODE': exit_doc['EXIT_CODE'], 'ERROR_CODE': exit_doc['ERROR_CODE']})
  except:
    pass
                       
except:
  try:
    err_desc = str(traceback.format_exc())
    raise uncatchException(err_desc)
  except uncatchException as cfe:
    audit_doc = OrderedDict()
    audit_rec=DriverEntryStreaming('streaming',envConfig['sourceName'],envConfig['prcs_runid'],'Failed',cfe,start_time,exit_doc['JSON'],log_file_name)
    audit_doc,exit_doc=utilsShared.gen_audit_dict_streaming(audit_doc,exit_doc,audit_rec,cfe)
    metadata_obj.insert_StreamingauditRecord(dbutils,envConfig,spark,audit_doc)
    exit_doc["EXIT_CODE"]=0
    logger.info('ERROR in  Processing file '+ str(envConfig['sourceName']) + '.'+ str(envConfig['tableName']) + '  for date '+ str(envConfig['batchDate']) + str(traceback.format_exc()))
    logger.info('Streaming - Failure Audit Process Completed')
    os.makedirs(log_file_folder, exist_ok=True)
    copyfile(local_log_file_name, log_file_name)
    try:
      print("++++++++++++!!!!!!++++++++++")
      #dbutils.notebook.exit({'AUDIT_RECORDS': exit_doc, 'EXIT_CODE': exit_doc['EXIT_CODE'], 'ERROR_CODE': exit_doc['ERROR_CODE']})
    except:
      pass  
  



2022-01-13 12:39:38,198 - utilsShared - INFO - Reading Env Config from: /dbfs/mnt/mountdatalake/AZ_IDFCodebase/conf/envConfig.yaml
2022-01-13 12:39:38,265 - utilsShared - INFO - Reading Env Config from: /dbfs/mnt/mountdatalake/AZ_IDFCodebase/conf/envConfig.yaml
2022-01-13 12:39:38,265 - utilsShared - INFO - Reading Env Config from: /dbfs/mnt/mountdatalake/AZ_IDFCodebase/conf/envConfig.yaml
2022-01-13 12:39:38,676 - smart_ingestion_driver - INFO - Performing Stream Ingestion Activity
2022-01-13 12:39:38,677 - smart_ingestion_driver - INFO - Since load date is not provided, considering load datetime as current/Execution timestamp
2022-01-13 12:39:38,677 - smart_ingestion_driver - INFO - /dbfs/mnt/mountdatalake/AZ_IDFCodebase/data-conf-streaming/code_test_streaming.json
2022-01-13 12:39:38,701 - smart_ingestion_driver - INFO - Reading config from: /dbfs/mnt/mountdatalake/AZ_IDFCodebase/data-conf-streaming/code_test_streaming.json
2022-01-13 12:39:38,701 - smart_ingestion_driver - INFO - Processing S1-ingestion-load
2022-01-13 12:39:38,701 - smart_ingestion_driver - DEBUG - multiple-streams-data-process
2022-01-13 12:39:41,174 - multiple-streams-data-process - INFO - Inside multiple-streams-data-process
2022-01-13 12:39:41,194 - multiple-streams-data-process - INFO - Reading from delta file
2022-01-13 12:39:41,194 - multiple-streams-data-process - INFO - /mnt/mountdatalake/idfdata/idfPOC/raw/dummydata10
2022-01-13 12:39:44,065 - multiple-streams-data-process - INFO - Read from delta topic
codetest
2022-01-13 12:39:44,065 - multiple-streams-data-process - INFO - Write the stream
2022-01-13 12:39:44,065 - multiple-streams-data-process - INFO - Checkpoint directory set at: /mnt/mountdatalake/idfdata/idfPOC/checkpoint/raw/dummydata10
2022-01-13 12:39:44,065 - multiple-streams-data-process - INFO - Default trigger stream will run micro-batch as soon as it can
2022-01-13 12:39:44,859 - multiple-streams-data-process - INFO - lastprogress: None
2022-01-13 12:39:55,992 - foreachbatch - INFO - Curent state: NEW
2022-01-13 12:39:55,993 - foreachbatch - INFO - epoch_id: 0
2022-01-13 12:39:55,993 - foreachbatch - INFO - foreachbatchid: ad11a75c-5739-4ec7-a060-dd06aa2b7da9
2022-01-13 12:40:03,987 - foreachbatch - INFO - RAW: files ingested at: /mnt/mountdatalake/idfdata/idfPOC/raw/dummydata01/

In [0]:
def delete_dir(dirname):
  files=dbutils.fs.ls(dirname)
  for f in files:
    if f.isDir():
      dbutils.fs.rm(f.path, recurse=True)

In [0]:
delete_dir('/mnt/mountdatalake/idfdata/idfPOC/checkpoint/raw/dummydata10')

In [0]:
print(log_file_name)

/dbfs/mnt/mountdatalake/idfdata/idfPOC/logs//code/test/2022-01-13/code_test_20220113_123938.log

In [0]:
newdf = spark.read.format("delta").load("/mnt/mountdatalake/idfdata/idfPOC/enrich/dummydata_enrich")
print(newdf.count())

5

In [0]:
display(newdf)

firstname middlename lastname dob gender salary Maria Anne Jones 39192 F 4000 Robert Williams 42114 M 4000 Michael Rose 40288 M 4000 James Smith 36636 M 3000 Jen Mary Brown F -1

In [0]:
newdf1 = spark.read.format("delta").load("/mnt/mountdatalake/idfdata/idfPOC/curated/impressions_curated5")
print(newdf1.count())

In [0]:
display(newdf1)

In [0]:
newdf1 = newdf1.select('itype','adId','cd')
newdf3 = newdf1.withColumn("src_Length_flag",when((newdf1.itype.isNotNull()) & (col("adId")>col("cd")),lit(1)).otherwise(lit(0)))
display(newdf3)

In [0]:
newdf2 = spark.read.format("delta").load("/mnt/mountdatalake/idfdata/idfPOC/enrich/impressions_enrich5")
print(newdf2.count())

In [0]:
display(newdf2)

In [0]:
data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df=spark.createDataFrame(data,columns)

In [0]:
display(df)

In [0]:
df.write.format("delta").save("/mnt/mountdatalake/idfdata/idfPOC/raw/dummydata10")

In [0]:
parDF1=spark.read.parquet("/mnt/mountdatalake/idfdata/idfPOC/raw/dummydata")

In [0]:
display(parDF1)

In [0]:
newdf2 = spark.read.format("delta").load("/mnt/mountdatalake/idfdata/idfPOC/raw/dummydata10")

In [0]:
display(newdf2)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import lit


def test_enrich_process():
    hash_table = [("test", 123445, "abc@gmail.com"),
             ("test", 56473, "dfegeg@gmail.com"),
             ("test",12337, "ab@gmail.com"),
             ("test", 123443435, "sumit@gmail.com")
             ]

    schema = StructType([
        StructField('Name', StringType(), True),
        StructField('Mobile', IntegerType(), True),
        StructField('Email', StringType(), True)
    ])

    spark = SparkSession.builder.getOrCreate()
    df = spark.createDataFrame(hash_table, schema)

    display(df)

    


In [0]:
test_enrich_process()

In [0]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import lit,col,trim
from pyspark_test import assert_pyspark_df_equal
from datetime import datetime


import json
 
# Opening JSON file
f = open('code_test_streaming.json')

objDict = json.load(f)

# print(objDict["S1-ingestion-load"])

def source_filter(spark,objDict,HASH_DF,batchDate):
	print("inside func")

	if "custom-query" in objDict:
		print("inside custom")
		HASH_DF = HASH_DF.where(objDict["custom-query"])
	else:
		print("inside else")
		objDict=objDict["S1-ingestion-load"]["data-enrich-process"]["filter"]

		for keyToProcess in sorted(objDict.keys()):
			print("inside for loop")
			processType = str(objDict[keyToProcess]["filter-type"])
			#logger.debug("Applying filter type : "+ processType)

			if processType == "notequal" :
				print("inside not equal")
				filterValue = str(objDict[keyToProcess]["value"]).strip()
				keyFilter = str(objDict[keyToProcess]["colName"])
				# logger.debug("Applying fiter type "+ processType + " on column :" + str(keyFilter))
				HASH_DF = HASH_DF.filter(trim(col(keyFilter)) != filterValue)

			# if processType == "like" :
			# 	filterValue = str(objDict[keyToProcess]["value"]).strip()
			# 	keyFilter = str(objDict[keyToProcess]["colName"])
			# 	logger.debug("Applying fiter type "+ processType + " on column :" + str(keyFilter))
			# 	HASH_DF = HASH_DF.filter(trim(col(keyFilter)).like(filterValue))

			# if processType == "equal" :
			# 	filterValue = str(objDict[keyToProcess]["value"]).strip()
			# 	keyFilter = str(objDict[keyToProcess]["colName"])
			# 	logger.debug("Applying fiter type "+ processType + " on column :" + str(keyFilter))
			# 	HASH_DF = HASH_DF.filter(trim(col(keyFilter)) == filterValue)

			# if processType == "col-equal" :
			# 	filterValue = str(objDict[keyToProcess]["col-to-match"]).strip()
			# 	keyFilter = str(objDict[keyToProcess]["colName"])
			# 	logger.debug("Applying fiter type "+ processType + " on column :" + str(keyFilter))
			# 	HASH_DF = HASH_DF.filter(trim(col(keyFilter)) == trim(col(filterValue)))

			# if processType == "null" :
			# 	keyFilter = str(objDict[keyToProcess]["colName"])
			# 	logger.debug("Applying fiter type "+ processType + " on column :" + str(keyFilter))
			# 	HASH_DF = HASH_DF.na.drop(subset = keyFilter)
			# 	HASH_DF = HASH_DF.filter(trim(col(keyFilter))!=lit(""))

			# if processType == "equal-list" :
			# 	filterList = objDict[keyToProcess]["value"]
			# 	keyFilter = str(objDict[keyToProcess]["colName"])
			# 	logger.debug("Applying fiter type "+ processType + " on column :" + str(keyFilter))
			# 	#print filterList
			# 	logger.debug("Filter List on  "+ str(filterList) )
			# 	HASH_DF = HASH_DF.filter(trim(col(keyFilter)).isin(filterList))

			# if processType == "max" :
			# 	keyFilter = str(objDict[keyToProcess]["colName"])
			# 	logger.debug("Applying max type "+ processType + " on column :" + str(keyFilter))
			# 	logger.debug("max logic started " + keyFilter)
			# 	HASH_DF.registerTempTable("maxTemp")
			# 	maxcol = HASH_DF.select(col(keyFilter)).distinct().collect()
			# 	logger.debug(maxcol)
			# 	HASH_DF = HASH_DF.where(str(keyFilter) + " = (select max("+ str(keyFilter) +") from maxTemp)")
			# 	maxcol = HASH_DF.select(str(keyFilter)).distinct().collect()
			# 	logger.debug(maxcol)
			# 	logger.debug("max logic ended")

			# if processType == "derived-filter":
			# 	colName=str(objDict[keyToProcess]["colName"])
			# 	suffix_final=""
			# 	print("suffix_final:"+suffix_final)
			# 	if "substarct-months" in objDict[keyToProcess]:
			# 		batch_date_formatted=datetime.strptime(batchDate,"%Y%m%d")-dateutil.relativedelta.relativedelta(months=objDict[keyToProcess]["substarct-months"])

			# 		month= batch_date_formatted.strftime("%b")
			# 		if "case" in objDict[keyToProcess]:
			# 			case = str(objDict[keyToProcess]["case"])
			# 			if case == "upper":
			# 				month = month.upper()
			# 			else:
			# 				month = month.lower()

			# 		suffix_final="%"+month+objDict[keyToProcess]["seprator"]+objDict[keyToProcess]["string-suffix"]

			# 		if "substarct-maxperiod" in objDict[keyToProcess]:
			# 			batch_date_formatted_per=datetime.strptime(batchDate,"%Y%m%d")-dateutil.relativedelta.relativedelta(months=objDict[keyToProcess]["substarct-maxperiod"])
			# 			period= str(batch_date_formatted_per.month)
			# 			suffix_final=month+objDict[keyToProcess]["seprator"]+objDict[keyToProcess]["string-suffix"]+period

			# 	if "substarct-periods" in objDict[keyToProcess]:
			# 		batch_date_formatted=datetime.strptime(batchDate,"%Y%m%d")-dateutil.relativedelta.relativedelta(months=objDict[keyToProcess]["substarct-periods"])
			# 		period= batch_date_formatted.strftime("%m")
			# 		if "string-prefix" in objDict[keyToProcess]:
			# 			string_prefix = str(objDict[keyToProcess]["string-prefix"])
			# 			suffix_final="%"+string_prefix+objDict[keyToProcess]["seprator"]+objDict[keyToProcess]["string-suffix"]+period+"%"
			# 		else:
			# 			suffix_final="%"+period+objDict[keyToProcess]["seprator"]+objDict[keyToProcess]["string-suffix"]+"%"

			# 	custom_string = "{} like '{}' ".format(colName, suffix_final)
			# 	print("suffix_final::"+suffix_final)
			# 	logger.debug("Applying fiter type "+ processType + " on column :" + str(colName))
			# 	HASH_DF = HASH_DF.where(custom_string)

	return HASH_DF



from pyspark.sql.types import StructField, StructType, StringType, IntegerType
from pyspark.sql import SparkSession

def test_filter():
    hash_table = [("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jane ","Anne","Ja","39192","F",4000),
              ("Katy ","Mani","Jones","39192","F",6000),
              ("Perry ","Anne","Jae","19192","F",23000),
              ("Ashley ","Swith","Cruze","49192","F",10000),
              ("Ricky ","Pery","Mily","59192","F",9000),
              ("Jen","Mary","Brown","","F",-1)]
    schema = StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
        StructField('lastname', StringType(), True),
        StructField('dob', StringType(), True),
        StructField('gender', StringType(), True),
        StructField('salary', IntegerType(), True)
    ])
    spark = SparkSession.builder.getOrCreate()
    df = spark.createDataFrame(hash_table, schema)
    final_df=source_filter(spark,objDict,df,'2022-02-01')
    final_df.show()
    hash_table1 = [("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
             ]
    schema1 = StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
        StructField('lastname', StringType(), True),
        StructField('dob', StringType(), True),
        StructField('gender', StringType(), True),
        StructField('salary', IntegerType(), True)
    ])
    spark1 = SparkSession.builder.getOrCreate()
    expected_df = spark1.createDataFrame(hash_table1, schema1)
    return final_df,expected_df
	
    	        
		
test_filter()
		


--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-746937768735915> in <module> 
 3 from pyspark . sql . types import StructField , StructType , StringType , IntegerType
 4 from pyspark . sql . functions import lit , col , trim
 ----> 5 from pyspark_test import assert_pyspark_df_equal
 6 from datetime import datetime
 7 

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 156 # Import the desired module. If you’re seeing this while debugging a failed import, 
 157 # look at preceding stack frames for relevant error information. 
 --> 158 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 159 
 160 is_root_import = thread_local . _nest_level == 1 

 ModuleNotFoundError : No module named 'pyspark_test'